Conectar drive personal

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importar librerías necesarias e instalar modelo para modelado de tópicos BERTopic

In [ ]:
import pandas as pd
!pip install bertopic

Importar modelo BERTopic e instanciarlo con parámetros de configuración básicos: lenguaje, cálculo de probabilidad de los tópicos obtenidos por documento y seguimiento de las fases del modelo (verbose)

In [ ]:
#IMPORTAR MODELO
from bertopic import BERTopic
topic_model = BERTopic(language="english", calculate_probabilities=False, verbose=True)

Primero se lee el dataset con los tweets limpios y pre-procesados. A continuación, se transforma la columna 'body' con los tweets a una lista de strings para utilizarla como entrada al modelo BERTopic. Finalmente, se obtienen los tópicos más relevantes del tópico

In [ ]:
#leer dataset
data_resto = pd.read_csv('/content/drive/MyDrive/TFG ADE/Data_Resto_Limpio.csv')
data=(data['body'].astype(str)).tolist()
#obtener topics iniciales
topics, _ = topic_model.fit_transform(data)

Batches:   0%|          | 0/5270 [00:00<?, ?it/s]

2023-11-13 17:00:55,002 - BERTopic - Transformed documents to Embeddings
2023-11-13 17:52:59,948 - BERTopic - Reduced dimensionality
2023-11-13 17:53:34,057 - BERTopic - Clustered reduced embeddings


Se obtiene información general sobre los tópicos obtenidos por el modelo: nombre/identificador, términos representativos y documentos representativos (-1 es el tópico outlier, no se tiene en cuenta en el análisis)

In [ ]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,49601,-1_good_retail_move_put,"[good, retail, move, put, 1800, break, earning...",[hard sold happens always recession turn trade...
1,0,2197,0_nan_usching_yoyo_quinnypig,"[nan, usching, yoyo, quinnypig, ching, goat, g...","[nan, nan, nan]"
2,1,2071,1_patent_research_method_image,"[patent, research, method, image, interface, s...",[patent 10311868 method apparatus using image ...
3,2,1887,2_guide_active_free_trade,"[guide, active, free, trade, nvda, surging, ac...","[active trade free trade guide nvda, active tr..."
4,3,1244,3_silver_commodity_gold_finance,"[silver, commodity, gold, finance, djia, econo...",[future djia stockmarket investing finance sto...
...,...,...,...,...,...
3040,3039,10,3039_facebooks_probe_virtual_mail,"[facebooks, probe, virtual, mail, photo, reali...",[article open data probe facebook latest relat...
3041,3040,10,3040_qqqkinda_qqqs_lowqqq_runqqq,"[qqqkinda, qqqs, lowqqq, runqqq, alltslaq, bun...","[qqqs getting taking heat ahead, leading qqqki..."
3042,3041,10,3041_compensation_haunting_excessive_siliconva...,"[compensation, haunting, excessive, siliconval...",[feel excessive stock compensation haunting ca...
3043,3042,10,3042_listening_soundbites_sound_karen,"[listening, soundbites, sound, karen, finerman...","[listening, listening, listening]"


Se extraen los n términos top del tópico más relevante/repetido (0) y la puntuación c-TF-IDF de cada uno

In [ ]:
topic_model.get_topic(0)

[('nan', 0.23847851626670571),
 ('usching', 0.002901807959728598),
 ('yoyo', 0.0025876484438826374),
 ('quinnypig', 0.0024041928116298227),
 ('ching', 0.002091550263011083),
 ('goat', 0.0016266449855116495),
 ('girl', 0.0011775685806509673),
 ('winner', 0.00027326061416764545),
 ('', 1e-05),
 ('', 1e-05)]

Debido al elevado número de tópicos encontrado (3044) y la limitación computacional, no es posible representar la distancia intertópica inicial

In [ ]:
#DISTANCIA INTERTOPICA INICIAL
topic_model.visualize_topics()

Se guarda el modelo para no perder la clasificación inicial cuando expire la sesión y se vuelve a cargar

In [ ]:
topic_model.save('/content/drive/MyDrive/TFG ADE/restotopicinicial')

In [ ]:
loaded_model = BERTopic.load('/content/drive/MyDrive/TFG ADE/restotopicinicial')

Ya que se obtienen un gran número de documentos pertenecientes al tópico -1 (outlier), se reducen los mismos y se actualizan las representaciones de los tópicos obtenidos

In [ ]:
new_topics = loaded_model.reduce_outliers(data, loaded_model.topics_)
loaded_model.update_topics(data, topics=new_topics)

Se agrupan los tópicos para obtener las relaciones existentes entre ellos y se visualiza la jerarquía de los 25 más relevantes, con el objetivo de reducir el número de tópicos

In [ ]:
#JERARQUIA INICIAL QUE AYUDA A REDUCIR EL NUMERO DE TOPICOS
loaded_model.visualize_hierarchy(top_n_topics=25)

Se van reduciendo el número de tópicos y visualizando la distancia intertópica hasta obtener el número óptimo (20) en el que la distancia es lo suficientemente grande como para que estos sean diferenciables

In [ ]:
#PRUEBAS TOPIC REDUCTION Y DISTANCIA INTERTOPICA RESULTANTE.
loaded_model.reduce_topics(data, nr_topics=22)
loaded_model.visualize_topics()

2023-11-15 22:02:43,301 - BERTopic - Reduced number of topics from 23 to 22


De nuevo se obtiene la información general sobre los 20 tópicos resultantes, actualizando la representación de tópicos del modelo aumentando el rango de N-Gramas entre 1 y 3. Se observa que el 1, 17 y 20 corresponden a acumulación de tweets "basura", por lo que no se tendrán en cuenta para el posterior análisis

In [ ]:
#informacion con reduccion de outliers
loaded_model.update_topics(data, n_gram_range=(1,3))
freq = loaded_model.get_topic_info()
freq

,Topic,Count,Name,Representation,Representative_Docs
0,-1,181,-1_350customers_c1300_gafamgooglaaplfbamznmsft...,"[350customers, c1300, gafamgooglaaplfbamznmsft...",[trumpchildrenshortfaangagaingooglamznaaplhowa...
1,0,164431,0_nflx_trade_stock_nvda,"[nflx, trade, stock, nvda, baba, option, marke...","[active trade free trade guide bidu nvda, acti..."
2,1,2197,1_nan_nan nan_nan nan nan_nan quinnypig goat,"[nan, nan nan, nan nan nan, nan quinnypig goat...","[nan, nan, nan]"
3,2,214,2_stock dailystkleadercount_dailystkleadercoun...,"[stock dailystkleadercount, dailystkleadercoun...","[leading stock dailystkleadercount, leading st..."
4,3,204,3_maxpain chart_option maxpain chart_option ma...,"[maxpain chart, option maxpain chart, option m...","[option maxpain chart free option, option maxp..."
5,4,182,4_scalp_crosshairs_profit_scalping,"[scalp, crosshairs, profit, scalping, nice, qu...","[scalp, scalp, scalp]"
6,5,179,5_consolidation_higher_break_year,"[consolidation, higher, break, year, level, mo...","[consolidation pattern break, consolidation hi..."
7,6,160,6_quantum_quantum computing_computing_quantumc...,"[quantum, quantum computing, computing, quantu...","[cloud quantum, quantum jimcramer opinion quan..."
8,7,148,7_linkedin_chainlink_link_azure,"[linkedin, chainlink, link, azure, azure linke...","[linkedin, linkedin, linkedin]"
9,8,105,8_cramer_jimcramer_jimcramer madmoneyoncnbc_ma...,"[cramer, jimcramer, jimcramer madmoneyoncnbc, ...",[alphabet stock cramer say jimcramer thestreet...


Se guarda el modelo con los 20 tópicos resultantes para no perderlo cuando expire la sesión y se vuelve a cargar

In [ ]:
loaded_model.save('/content/drive/MyDrive/TFG ADE/restotopic22')

In [ ]:
loaded_model=BERTopic.load('/content/drive/MyDrive/TFG ADE/restotopic22')

Se actualizan las etiquetas estándar de los tópicos resultantes asignadas por el modelo a unas etiquetas custom que faciliten la interpretabilidad de los mismos

In [ ]:
#custom labels -> no se tiene en cuenta 2, 17 y 20
labels_resto={0: "Inversiones y comparativa entre empresas",
        1: "Sin identificar",
        2: "Top acciones",
        3: "Opciones y maxpain",
        4: "Técnica scalping",
        5: "Consolidación acciones",
        6:"Computación cuántica",
              7:"Otras empresas",
              8:"Análisis CNBC",
              9:"Destitución Trump",
              10:"Volatilidad",
              11:"Inversión en empresas",
              12:"Periodos temporales",
              13:"Auriculares",
              14:"Código",
              15:"Roku",
              16: "Otros",
              17:"Sin identificar",
              18:"Preapertura mercado",
              19:"Trackerbot",
              20:"Sin identificar"}
loaded_model.set_topic_labels(labels_resto)

Se representan en gráficos de barras los 18 tópicos ordenados por relevancia, en los cuales se obtienen los términos más repetidos por tópico y su puntuación c-TF-IDF

In [ ]:
loaded_model.visualize_barchart(topics=[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19], width=600, custom_labels=True)

Se representa la matriz de similitud entre los 10 primeros tópicos más relevantes, para comprobar si son lo suficientemente distintos (valores de similitud menor o igual a 0.5)

In [ ]:
loaded_model.visualize_heatmap(n_clusters=5, topics=[0,2,3,4,5,6,7,8,9,10],custom_labels=True, width=1200,height=1000)

Se añaden las columnas con la categorización por tópico (número y etiqueta custom) y las palabras clave de cada uno en el dataset

In [ ]:
data['Tópico']=loaded_model.get_document_info(data['body'])['Topic']
data['Categoría']=loaded_model.get_document_info(data['body'])['CustomName']
data['Palabras clave']=loaded_model.get_document_info(data['body'])['Top_n_words']

Se descartan los tweets que tratan sobre el tópico -1 (outlier), y los que acumulan tweets "basura" (1,17,20)

In [ ]:
data.drop(data[(data['Tópico'] ==-1)].index, inplace=True)
data.drop(data[(data['Tópico'] ==1)].index, inplace=True)
data.drop(data[(data['Tópico'] ==17)].index, inplace=True)
data.drop(data[(data['Tópico'] ==20)].index, inplace=True)

Se guarda el dataset con las nuevas columnas

In [ ]:
#se guarda para el analisis de sentimientos
data.to_csv('/content/drive/MyDrive/TFG ADE/RestoData22Topics.csv')

Se obtiene la variación frecuencia por tópico a lo largo de la franja temporal de estudio

In [ ]:
data['post_date_only']= pd.to_datetime(data['post_date']).dt.date

topics_over_time = loaded_model.topics_over_time(data['body'].astype(str), data['post_date_only'])

In [ ]:
loaded_model.visualize_topics_over_time(topics_over_time, topics=[0, 2, 3, 4,5], custom_labels=True, width=1500)

In [ ]:
loaded_model.visualize_topics_over_time(topics_over_time, topics=[6, 7, 8, 9,10,11,12], custom_labels=True, width=1500)

In [ ]:
loaded_model.visualize_topics_over_time(topics_over_time, topics=[13, 14, 15, 16,18,19], custom_labels=True, width=1500)

Se exporta la evolución temporal por tópico para representarla en otra herramienta

In [ ]:
datos= pd.read_csv('/content/drive/MyDrive/TFG ADE/RestoData22Topics.csv')

# evolucion temporal topicos
datos['post_date'] = pd.to_datetime(datos['post_date'])
topics = [0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19]

for topic in topics:
  topic_tweets = datos[datos['Tópico'] == topic]

  # Group the data by day and count the number of tweets per day
  t_per_day = topic_tweets.resample('D', on='post_date').count()
  columns_to_delete = ['body',  'Categoría', 'Palabras clave', 'Unnamed: 0', 'tweet_id']
  t_per_day = t_per_day.drop(columns=columns_to_delete)
  t_per_day = t_per_day.rename(columns={'Tópico': f'Tweets diarios sobre tópico {topic}'})
  t_per_day.to_csv(f'drive/MyDrive/TFG ADE/resto{topic}.csv')